# Prosty Klasyfikator Obrazów

Ten notebook pokazuje, jak klasyfikować obrazy za pomocą wstępnie wytrenowanej sieci neuronowej.

**Czego się nauczysz:**
- Jak załadować i używać wstępnie wytrenowanego modelu
- Przetwarzanie obrazów
- Dokonywanie predykcji na obrazach
- Zrozumienie wyników pewności

**Przypadek użycia:** Identyfikacja obiektów na obrazach (np. "kot", "pies", "samochód" itd.)

---


## Krok 1: Importowanie wymaganych bibliotek

Zaimportujmy potrzebne narzędzia. Nie martw się, jeśli jeszcze nie rozumiesz wszystkich!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Krok 2: Załaduj wstępnie wytrenowany model

Użyjemy **MobileNetV2**, sieci neuronowej już wytrenowanej na milionach obrazów.

To nazywa się **Transfer Learning** - korzystanie z modelu wytrenowanego przez kogoś innego!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Krok 3: Funkcje pomocnicze

Stwórzmy funkcje do ładowania i przygotowywania obrazów dla naszego modelu.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Krok 4: Testowanie na przykładowych obrazach

Spróbujmy sklasyfikować kilka obrazów z internetu!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klasyfikuj Każdy Obraz


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Krok 5: Wypróbuj własne obrazy!

Zamień poniższy URL na dowolny URL obrazu, który chcesz sklasyfikować.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Co się właśnie wydarzyło?

1. **Załadowaliśmy wstępnie wytrenowany model** - MobileNetV2 został wytrenowany na milionach obrazów
2. **Przetworzyliśmy obrazy** - Zmieniliśmy ich rozmiar i sformatowaliśmy je dla modelu
3. **Model dokonał predykcji** - Wyprowadził prawdopodobieństwa dla 1000 klas obiektów
4. **Zdekodowaliśmy wyniki** - Przekształciliśmy liczby na czytelne dla człowieka etykiety

### Zrozumienie wyników pewności

- **90-100%**: Bardzo pewny (niemal na pewno poprawny)
- **70-90%**: Pewny (prawdopodobnie poprawny)
- **50-70%**: Średnio pewny (może być poprawny)
- **Poniżej 50%**: Mało pewny (niepewny)

### Dlaczego predykcje mogą być błędne?

- **Nietypowy kąt lub oświetlenie** - Model był trenowany na typowych zdjęciach
- **Wiele obiektów** - Model oczekuje jednego głównego obiektu
- **Rzadkie obiekty** - Model zna tylko 1000 kategorii
- **Niska jakość obrazu** - Rozmazane lub pikselowe obrazy są trudniejsze do analizy

---


## 🚀 Kolejne kroki

1. **Wypróbuj różne obrazy:**
   - Znajdź obrazy na [Unsplash](https://unsplash.com)
   - Kliknij prawym przyciskiem myszy → „Kopiuj adres obrazu”, aby uzyskać URL

2. **Eksperymentuj:**
   - Co się dzieje z abstrakcyjną sztuką?
   - Czy potrafi rozpoznać obiekty z różnych perspektyw?
   - Jak radzi sobie z wieloma obiektami?

3. **Dowiedz się więcej:**
   - Odkryj [lekcje o widzeniu komputerowym](../lessons/4-ComputerVision/README.md)
   - Naucz się trenować własny klasyfikator obrazów
   - Zrozum, jak działają CNN (Konwolucyjne Sieci Neuronowe)

---

## 🎉 Gratulacje!

Właśnie stworzyłeś klasyfikator obrazów, wykorzystując najnowocześniejszą sieć neuronową!

Ta sama technika jest wykorzystywana w:
- Google Photos (organizacja zdjęć)
- Samochodach autonomicznych (rozpoznawanie obiektów)
- Diagnostyce medycznej (analiza zdjęć rentgenowskich)
- Kontroli jakości (wykrywanie wad)

Kontynuuj eksplorację i naukę! 🚀



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
